In [1]:
import pandas as pd
import os
import shutil
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

from QC.utils import shell_do

Plink is found
Plink2 is found


In [2]:
############ SET THIS EVERY RELEASE ############
release = '5'

key_file = f'/data/GP2/clinical/master_key/GP2_master_key_FINAL_release5_n26728.txt'
key = pd.read_csv(f'{key_file}')
key.loc[:,'IID'] = key.loc[:,'SentrixBarcode_A'].astype(str) + '_' + key.loc[:,'SentrixPosition_A'].astype(str)
key.loc[:,'FID'] = "0"


idat_path = '/data/GP2/raw_genotypes/idats/'
swarm_scripts_dir = f'/data/GP2/users/vitaled2/swarm'

ilmn_files_path = '/data/GP2/utils/ilmn_files'
# A1 is hg19, A2 is hg38. csv needed for indel calls
bpm_csv = f'{ilmn_files_path}/NeuroBooster_20042459_A2.csv' 
bpm = f'{ilmn_files_path}/NeuroBooster_20042459_A2.bpm'
egt = f'{ilmn_files_path}/recluster_09272022.egt'
iaap = f'{ilmn_files_path}/iaap-cli/iaap-cli'

ref_fasta = '/data/vitaled2/ref/GRCh38/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna'

chroms = [str(i) for i in range(1,23)] + ['X','Y','M']

samples_list = key.loc[:,'filename'].unique()
barcodes_list = list(set([x.split('_')[0] for x in samples_list]))

################################################################################################################

# genes = pd.read_csv('/data/GP2/utils/ref_dir/glist-hg38', sep='\s+', header=None, names=['chr','start','end','symbol'], dtype={'chr':str,'start':int,'end':int})
# genes.columns = ['CHR','START','STOP','NAME']
# genes_out = genes.loc[genes.CHR.isin(chroms)]
# genes_out[['NAME','CHR','START','STOP']].to_csv('/data/GP2/utils/ref_dir/glist_hg38_intervals.csv', index=False)
# # gene_list originally from plink glist
# gene_list = '/data/GP2/utils/ref_dir/glist_hg38_intervals.csv'
# !cp /data/CARD/PD/GP2/ref_panel/glist_hg38_chr*.csv /data/GP2/utils/ref_dir/

raw_geno_path = '/data/GP2/raw_genotypes'
snp_metrics_path = f'{raw_geno_path}/snp_metrics'
idat_path = f'{raw_geno_path}/idats'
plink_file_path = f'{raw_geno_path}/ped_bed'

In [3]:
key

,GP2ID,GP2sampleID,manifest_id,phenotype,pheno_for_qc,other_pheno,sex_for_qc,age,age_of_onset,age_at_diagnosis,...,study,pruned,pruned_reason,label,related,SentrixBarcode_A,SentrixPosition_A,filename,IID,FID
0,IMMUNEPD_000001,IMMUNEPD_000001_s1,m1,PD,2,PD,1,61.0,NaN,61.0,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R01C01,206966380021_R01C01,206966380021_R01C01,0
1,IMMUNEPD_000002,IMMUNEPD_000002_s1,m1,PD,2,PD,1,66.0,NaN,66.0,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R02C01,206966380021_R02C01,206966380021_R02C01,0
2,IMMUNEPD_000003,IMMUNEPD_000003_s1,m1,Control,1,Control,2,55.0,NaN,NaN,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R03C01,206966380021_R03C01,206966380021_R03C01,0
3,IMMUNEPD_000004,IMMUNEPD_000004_s1,m1,Control,1,Control,1,50.0,NaN,NaN,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R04C01,206966380021_R04C01,206966380021_R04C01,0
4,IMMUNEPD_000005,IMMUNEPD_000005_s1,m1,Control,1,Control,2,74.0,NaN,NaN,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R05C01,206966380021_R05C01,206966380021_R05C01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26723,MDGAP-EBB_000149,MDGAP-EBB_000149_s1,m4,Control,1,Control,1,NaN,NaN,NaN,...,MDGAP-EBB,1,callrate_prune,NaN,NaN,206412050061,R04C01,206412050061_R04C01,206412050061_R04C01,0
26724,MDGAP-EBB_000150,MDGAP-EBB_000150_s1,m4,Control,1,Control,2,NaN,NaN,NaN,...,MDGAP-EBB,1,callrate_prune,NaN,NaN,206412050061,R05C01,206412050061_R05C01,206412050061_R05C01,0
26725,MDGAP-EBB_000151,MDGAP-EBB_000151_s1,m4,Control,1,Control,1,NaN,NaN,NaN,...,MDGAP-EBB,1,callrate_prune,NaN,NaN,206412050061,R06C01,206412050061_R06C01,206412050061_R06C01,0
26726,MDGAP-EBB_000152,MDGAP-EBB_000152_s1,m4,Control,1,Control,2,NaN,NaN,NaN,...,MDGAP-EBB,1,callrate_prune,NaN,NaN,206412050061,R07C01,206412050061_R07C01,206412050061_R07C01,0


In [54]:
# get snp metrics
metrics_exist_list = []
bcftools_plugins_path = 'bin'
with open(f'{swarm_scripts_dir}/snp_metrics.swarm', 'w') as f:
    for code in barcodes_list:
        idat_path_ = f'{idat_path}/{code}'
        metrics_out = f'{snp_metrics_path}'
        # metrics_code_path = f'{snp_metrics_path}/{code}'
        cmd = f'\
python run_snp_metrics_pipeline.py \
--idat_path {idat_path_} \
--bpm {bpm} \
--bpm_csv {bpm_csv} \
--egt {egt} \
--ref_fasta {ref_fasta} \
--out_path {metrics_out} \
--iaap {iaap} \
--bcftools_plugins_path {bcftools_plugins_path}'
        f.write(f'{cmd}\n')
f.close()


In [18]:
!swarm -f {swarm_scripts_dir}/snp_metrics.swarm -g 16 -t 16 --time=04:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

63708079


In [6]:
#check if output for chr2 exists for all samples 
exist_metrics = []
missing_metrics = []

for sample in samples_list:
    code = sample.split('_')[0]
    metrics_file = f'/data/GP2/raw_genotypes/snp_metrics/{code}/snp_metrics_{sample}_chr2.parquet'
    
    if os.path.isfile(metrics_file):
        exist_metrics.append(sample)
    else:
        missing_metrics.append(sample)

chroms = [x for x in range(1,23)]
exist_chrom = []
missing_chrom = []
# check if all chromosomes exist for samples for which chr2 exists
for sample in exist_metrics:
    code = sample.split('_')[0]
    for chrom in chroms:
        
        metrics_path = f'/data/GP2/raw_genotypes/snp_metrics/{code}/snp_metrics_{sample}_chr{chrom}.parquet'
        
        if os.path.isfile(metrics_path):
            exist_chrom.append(metrics_path)
        else:
            missing_chrom.append(metrics_path)
    
    

        
print(f'chr2 metrics exist: {len(exist_metrics)}')
print(f'chr2 missing metrics: {len(missing_metrics)}')
print(f'samples with all chroms: {len(exist_chrom)/22}')
print(f'chr files missing from samples with chr2 {len(missing_chrom)}')

chr2 metrics exist: 26578
chr2 missing metrics: 150
samples with all chroms: 26578.0
chr files missing from samples with chr2 0


In [7]:
# now check if we have idats for missing snp metrics
missing_idats = []
exists_idats = []
for sample in missing_metrics:
    code = sample.split('_')[0]
    grn = f'{idat_path}/{code}/{sample}_Grn.idat'
    red = f'{idat_path}/{code}/{sample}_Red.idat'
    if os.path.isfile(grn) & os.path.isfile(red):
        exists_idats.append(sample)
    else:
        missing_idats.append(sample)

In [8]:
print(len(missing_idats))
print(len(exists_idats))

93
57


In [4]:
cnv_path = f'/data/GP2/raw_genotypes/cnvs/release{release}'
covar_path = f'{cnv_path}/key/release{release}_covar.csv'
# using just autosomes for CNV analysis due to call quality
chroms = [str(i) for i in range(1,23)]

ancestry_labels = [x.split('/')[-1].replace('.bed','').split('_')[-1] for x in glob.glob(f'/data/GP2/quality_control/release{release}/genotype_qc/GP2_round5_APRIL_2023_*.bed') if '_maf_hwe' not in x]

key[['FID','GP2sampleID']].to_csv(f'{cnv_path}/key/release{release}.samples', sep='\t', header=False, index=False)
key[['GP2sampleID','IID']].to_csv(f'{cnv_path}/key/release{release}_sample_id_key.csv')
release_covars = key.loc[:,['FID', 'GP2sampleID','sex_for_qc', 'age', 'age_of_onset']]
release_covars.to_csv(covar_path)

cnv_types = ['PERCENT_BAF_INSERTION','PERCENT_L2R_DELETION','PERCENT_L2R_DUPLICATION']

In [13]:
with open(f'{swarm_scripts_dir}/cnv_geno_qc.swarm', 'w') as f:

    for label in ancestry_labels:
        geno_name = f'GP2_round5_APRIL_2023_{label}'
        out_path = f'{cnv_path}/genotypes/{geno_name}_clean'
        geno_path = f'/data/GP2/quality_control/release{release}/genotype_qc/{geno_name}'
    
        cmd = f'\
python run_cnv_qc_pipeline.py \
--geno_path {geno_path} \
--out_path {out_path} \
--covar_path {covar_path} \
--maf 0.01 \
--geno 0.2 \
--hwe 5e-6 \
--indep_pairwise 1000 10 0.01 \
--samples_path {cnv_path}/key/release{release}.samples'
        
        f.write(f'{cmd}\n')
f.close()
!cat {swarm_scripts_dir}/cnv_geno_qc.swarm


python run_cnv_qc_pipeline.py --geno_path /data/GP2/quality_control/release5/genotype_qc/GP2_round5_APRIL_2023_FIN --out_path /data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_FIN_clean --covar_path /data/GP2/raw_genotypes/cnvs/release5/key/release5_covar.csv --maf 0.01 --geno 0.2 --hwe 5e-6 --indep_pairwise 1000 10 0.01 --samples_path /data/GP2/raw_genotypes/cnvs/release5/key/release5.samples
python run_cnv_qc_pipeline.py --geno_path /data/GP2/quality_control/release5/genotype_qc/GP2_round5_APRIL_2023_EUR --out_path /data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_EUR_clean --covar_path /data/GP2/raw_genotypes/cnvs/release5/key/release5_covar.csv --maf 0.01 --geno 0.2 --hwe 5e-6 --indep_pairwise 1000 10 0.01 --samples_path /data/GP2/raw_genotypes/cnvs/release5/key/release5.samples
python run_cnv_qc_pipeline.py --geno_path /data/GP2/quality_control/release5/genotype_qc/GP2_round5_APRIL_2023_EAS --out_path /data/GP2/raw_genotypes/cnvs/release5/ge

In [28]:
!swarm -f {swarm_scripts_dir}/cnv_geno_qc.swarm -g 16 -t 16 --time=02:00:00 --logdir {swarm_scripts_dir}/logs --module python/3.8,plink/1.9 --gres=lscratch:20 --partition=norm

63958236


In [16]:
# check outputs from qc pipeline

for label in ancestry_labels:
    extensions = ['fam','cov','eigenvec','bim', 'snps']
    out_geno = f'/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_{label}_clean.bed'
    print(f'{out_geno}')
    for ext in extensions:
        out_geno_path = f'/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_{label}_clean.{ext}'
        if os.path.isfile(out_geno):
            count = !cat {out_geno_path} | wc -l
            print(f'{ext}: {count}') 
            
        else:
            print(f'{out_geno} does not exist!!!')
    print()
    print()


/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_FIN_clean.bed
fam: ['16']
cov: ['16']
eigenvec: ['16']
bim: ['1152']
snps: ['1152']


/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_EUR_clean.bed
fam: ['15356']
cov: ['15356']
eigenvec: ['15356']
bim: ['14573']
snps: ['14573']


/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_EAS_clean.bed
fam: ['2913']
cov: ['2913']
eigenvec: ['2913']
bim: ['17255']
snps: ['17255']


/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_AAC_clean.bed
fam: ['1331']
cov: ['1331']
eigenvec: ['1331']
bim: ['13110']
snps: ['13110']


/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_AJ_clean.bed
fam: ['1546']
cov: ['1546']
eigenvec: ['1546']
bim: ['4747']
snps: ['4747']


/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_MDE_clean.bed
fam: ['149']
cov: ['149']
eigenvec: ['149']
bim: ['2672']
snps: ['2672']


/data/GP2/raw_genotypes/cnvs/

In [19]:

total_sample_n = 0
for label in ancestry_labels:
    out_name = f'GP2_round5_APRIL_2023_{label}_clean'
    out_path = f'{cnv_path}/genotypes/{out_name}'
    fam = pd.read_csv(f'{out_path}.fam', sep='\s+', header=None, names=['fid','iid','pat','mat','sex','pheno'])
    pheno_counts = fam.pheno.value_counts()
    ncases = pheno_counts[2]
    ncontrols = pheno_counts[1]
    ntotal = ncases + ncontrols
    total_sample_n += ntotal
    nsnps = !cat {out_path}.bim | wc -l
    print(f'{label} | total_n: {ntotal} | n_cases: {ncases} | n_controls: {ncontrols} | n_snps: {int(nsnps[0])}')
    print()

FIN | total_n: 13 | n_cases: 10 | n_controls: 3 | n_snps: 1152

EUR | total_n: 14196 | n_cases: 9230 | n_controls: 4966 | n_snps: 14573

EAS | total_n: 2900 | n_cases: 878 | n_controls: 2022 | n_snps: 17255

AAC | total_n: 1320 | n_cases: 289 | n_controls: 1031 | n_snps: 13110

AJ | total_n: 1126 | n_cases: 740 | n_controls: 386 | n_snps: 4747

MDE | total_n: 124 | n_cases: 106 | n_controls: 18 | n_snps: 2672

SAS | total_n: 215 | n_cases: 80 | n_controls: 135 | n_snps: 3273

CAS | total_n: 474 | n_cases: 195 | n_controls: 279 | n_snps: 5367

AMR | total_n: 550 | n_cases: 345 | n_controls: 205 | n_snps: 5020

AFR | total_n: 2343 | n_cases: 855 | n_controls: 1488 | n_snps: 18843



In [24]:
key

,GP2ID,GP2sampleID,manifest_id,phenotype,pheno_for_qc,other_pheno,sex_for_qc,age,age_of_onset,age_at_diagnosis,...,study,pruned,pruned_reason,label,related,SentrixBarcode_A,SentrixPosition_A,filename,IID,FID
0,IMMUNEPD_000001,IMMUNEPD_000001_s1,m1,PD,2,PD,1,61.0,NaN,61.0,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R01C01,206966380021_R01C01,206966380021_R01C01,0
1,IMMUNEPD_000002,IMMUNEPD_000002_s1,m1,PD,2,PD,1,66.0,NaN,66.0,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R02C01,206966380021_R02C01,206966380021_R02C01,0
2,IMMUNEPD_000003,IMMUNEPD_000003_s1,m1,Control,1,Control,2,55.0,NaN,NaN,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R03C01,206966380021_R03C01,206966380021_R03C01,0
3,IMMUNEPD_000004,IMMUNEPD_000004_s1,m1,Control,1,Control,1,50.0,NaN,NaN,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R04C01,206966380021_R04C01,206966380021_R04C01,0
4,IMMUNEPD_000005,IMMUNEPD_000005_s1,m1,Control,1,Control,2,74.0,NaN,NaN,...,IMMUNEPD,0,NaN,EUR,NaN,206966380021,R05C01,206966380021_R05C01,206966380021_R05C01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26723,MDGAP-EBB_000149,MDGAP-EBB_000149_s1,m4,Control,1,Control,1,NaN,NaN,NaN,...,MDGAP-EBB,1,callrate_prune,NaN,NaN,206412050061,R04C01,206412050061_R04C01,206412050061_R04C01,0
26724,MDGAP-EBB_000150,MDGAP-EBB_000150_s1,m4,Control,1,Control,2,NaN,NaN,NaN,...,MDGAP-EBB,1,callrate_prune,NaN,NaN,206412050061,R05C01,206412050061_R05C01,206412050061_R05C01,0
26725,MDGAP-EBB_000151,MDGAP-EBB_000151_s1,m4,Control,1,Control,1,NaN,NaN,NaN,...,MDGAP-EBB,1,callrate_prune,NaN,NaN,206412050061,R06C01,206412050061_R06C01,206412050061_R06C01,0
26726,MDGAP-EBB_000152,MDGAP-EBB_000152_s1,m4,Control,1,Control,2,NaN,NaN,NaN,...,MDGAP-EBB,1,callrate_prune,NaN,NaN,206412050061,R07C01,206412050061_R07C01,206412050061_R07C01,0


In [30]:
for label in ['EUR']:
    out_name = f'GP2_round5_APRIL_2023_{label}_clean'
    geno_path = f'{cnv_path}/genotypes/{out_name}'
    bim_path = f'{geno_path}.bim'
    fam = pd.read_csv(f'{geno_path}.fam', sep='\s+', header=None, names=['FID','IID','pat','mat','sex','pheno'])
    fam_key = fam.merge(key, left_on='IID', right_on='GP2sampleID', how='left')
    
    # samples = pd.read_csv(f'{geno_path}_barcode.samples', header=None, names=['IID'])
    label_dir = f'{cnv_path}/cnv_calls/{label}'
    os.makedirs(label_dir, exist_ok=True)
    for sample in fam_key.filename.unique():
        code = sample.split('_')[0]
        for chrom in chroms:

            mfile = f'{snp_metrics_path}/{code}/snp_metrics_{sample}_chr{chrom}.csv'

            cnv_out = f'{label_dir}/CNV_{label}_{sample}_chr{chrom}.csv'
            intervals = f'/data/GP2/utils/ref_dir/glist_hg38_chr{chrom}.csv'

            cmd = f'\
python run_cnv_pipeline.py \
--metrics {mfile} \
--bim {bim_path} \
--out_path {cnv_out} \
--intervals {intervals} \
--min_variants 10 \
--kb_window 250 \
--min_gentrain 0.2'

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [28]:
fam_key

,FID_x,IID_x,pat,mat,sex,pheno,GP2ID,GP2sampleID,manifest_id,phenotype,...,study,pruned,pruned_reason,label,related,SentrixBarcode_A,SentrixPosition_A,filename,IID_y,FID_y
0,0,APGS_000001_s1,0,0,1,2,APGS_000001,APGS_000001_s1,m1,PD,...,APGS,0,NaN,EUR,NaN,206677820008,R01C01,206677820008_R01C01,206677820008_R01C01,0
1,0,APGS_000002_s1,0,0,1,2,APGS_000002,APGS_000002_s1,m1,PD,...,APGS,0,NaN,EUR,NaN,206677820008,R02C01,206677820008_R02C01,206677820008_R02C01,0
2,0,APGS_000003_s1,0,0,1,2,APGS_000003,APGS_000003_s1,m1,PD,...,APGS,0,NaN,EUR,NaN,206677820008,R03C01,206677820008_R03C01,206677820008_R03C01,0
3,0,APGS_000004_s1,0,0,2,2,APGS_000004,APGS_000004_s1,m1,PD,...,APGS,0,NaN,EUR,1.0,206677820008,R04C01,206677820008_R04C01,206677820008_R04C01,0
4,0,APGS_000005_s1,0,0,1,2,APGS_000005,APGS_000005_s1,m1,PD,...,APGS,0,NaN,EUR,NaN,206677820008,R05C01,206677820008_R05C01,206677820008_R05C01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15351,0,UMD_000475_s1,0,0,1,2,UMD_000475,UMD_000475_s1,m1,PD,...,UMD,0,NaN,EUR,NaN,204701870092,R04C01,204701870092_R04C01,204701870092_R04C01,0
15352,0,UMD_000476_s1,0,0,1,1,UMD_000476,UMD_000476_s1,m1,Control,...,UMD,0,NaN,EUR,NaN,204701870051,R05C01,204701870051_R05C01,204701870051_R05C01,0
15353,0,UMD_000477_s1,0,0,2,2,UMD_000477,UMD_000477_s1,m1,PD,...,UMD,0,NaN,EUR,NaN,204701870092,R02C01,204701870092_R02C01,204701870092_R02C01,0
15354,0,UMD_000478_s1,0,0,1,1,UMD_000478,UMD_000478_s1,m1,Control,...,UMD,0,NaN,EUR,NaN,204701860092,R05C01,204701860092_R05C01,204701860092_R05C01,0


In [42]:

with open(f'{swarm_scripts_dir}/cnvs.swarm', 'w') as f:
    for label in ancestry_labels:
    # for label in ['EUR']:
        out_name = f'GP2_round5_APRIL_2023_{label}_clean'
        geno_path = f'{cnv_path}/genotypes/{out_name}'
        bim_path = f'{geno_path}.bim'
        fam = pd.read_csv(f'{geno_path}.fam', sep='\s+', header=None, names=['FID','IID','pat','mat','sex','pheno'])
        fam_key = fam.merge(key, left_on='IID', right_on='GP2sampleID', how='left')

        # samples = pd.read_csv(f'{geno_path}_barcode.samples', header=None, names=['IID'])
        label_dir = f'{cnv_path}/cnv_calls/{label}'
        os.makedirs(label_dir, exist_ok=True)
        for sample in fam_key.filename.unique():
            code = sample.split('_')[0]
            for chrom in chroms:

                mfile = f'{snp_metrics_path}/{code}/snp_metrics_{sample}_chr{chrom}.parquet'

                cnv_out = f'{label_dir}/CNV_{label}_{sample}_chr{chrom}.parquet'
                intervals = f'/data/GP2/utils/ref_dir/glist_hg38_chr{chrom}.csv'

                cmd = f'\
python run_cnv_pipeline.py \
--metrics {mfile} \
--bim {bim_path} \
--out_path {cnv_out} \
--intervals {intervals} \
--min_variants 10 \
--kb_window 250 \
--min_gentrain 0.2'
            
                f.write(f'{cmd}\n')
f.close()

In [55]:
!swarm -f {swarm_scripts_dir}/cnvs.swarm -g 4 -t 8 --time=00:15:00 --logdir {swarm_scripts_dir}/logs --module python/3.8 --gres=lscratch:20 --partition=norm

64115371


In [52]:
# !cat {swarm_scripts_dir}/logs/swarm_64098176_989.e
# !cat {swarm_scripts_dir}/cnvs.swarm | grep EUR
!python run_cnv_pipeline.py --metrics /data/GP2/raw_genotypes/snp_metrics/206677820008/snp_metrics_206677820008_R01C01_chr1.parquet --bim /data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_EUR_clean.bim --out_path /data/GP2/raw_genotypes/cnvs/release5/cnv_calls/EUR/CNV_EUR_206677820008_R01C01_chr1.parquet --intervals /data/GP2/utils/ref_dir/glist_hg38_chr1.csv --min_variants 10 --kb_window 250 --min_gentrain 0.2

Plink is found
Plink2 is found


In [53]:
test = pd.read_parquet('/data/GP2/raw_genotypes/cnvs/release5/cnv_calls/EUR/CNV_EUR_206677820008_R01C01_chr1.parquet')
test.head()

,INTERVAL,NUM_VARIANTS,PERCENT_BAF_INSERTION,PERCENT_L2R_DELETION,PERCENT_L2R_DUPLICATION,START_PLUS_WINDOW,START,STOP,STOP_PLUS_WINDOW
0,A3GALT2,0,NaN,NaN,NaN,33056765,33306765,33321098,33571098
1,AADACL3,0,NaN,NaN,NaN,12466114,12716114,12728759,12978759
2,AADACL4,0,NaN,NaN,NaN,12394546,12644546,12667086,12917086
3,ABCA4,0,NaN,NaN,NaN,93742837,93992837,94121149,94371149
4,ABCB10,0,NaN,NaN,NaN,229266581,229516581,229558695,229808695


In [64]:
intervals_df

,CHR,START,STOP,NAME
0,1,33306765,33321098,A3GALT2
1,1,12716114,12728759,AADACL3
2,1,12644546,12667086,AADACL4
3,1,93992837,94121149,ABCA4
4,1,229516581,229558695,ABCB10
...,...,...,...,...
2807,1,33472630,33496395,ZSCAN20
2808,1,45016403,45206578,ZSWIM5
2809,1,52842510,52894575,ZYG11A
2810,1,52726458,52827341,ZYG11B


In [76]:
kb_window=250
min_variants = 10
min_gentrain = 0.2

metrics_df = pd.read_parquet('/data/GP2/raw_genotypes/snp_metrics/206677820008/snp_metrics_206677820008_R01C01_chr1.parquet')
bim = pd.read_csv('/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_EUR_clean.bim', sep='\s+', header=None, names=['chr','id','cm','bp','a1','a2'], usecols=['id'])
intervals_df = pd.read_csv('/data/GP2/utils/ref_dir/glist_hg38_chr1.csv', dtype={'CHR':str})
sample_df = metrics_df.loc[(metrics_df.snpID.isin(bim.id)) & (metrics_df.GenTrain_Score>=min_gentrain)]

# Merge the dataframes
sample_df = sample_df.merge(intervals_df, left_on='chromosome', right_on='CHR', suffixes=('', '_interval'))

# Calculate start and stop plus window
sample_df['START_PLUS_WINDOW'] = sample_df['START'] - kb_window * 1000
sample_df['STOP_PLUS_WINDOW'] = sample_df['STOP'] + kb_window * 1000

# Filter the data
sample_df = sample_df[(sample_df['position'] >= sample_df['START_PLUS_WINDOW']) & (sample_df['position'] <= sample_df['STOP_PLUS_WINDOW'])]



# Calculate new columns
sample_df['BAF_insertion'] = ((sample_df['BAlleleFreq'].between(0.65, 0.85, inclusive='neither')) | (sample_df['BAlleleFreq'].between(0.15, 0.35, inclusive='neither'))).astype(int)
sample_df['L2R_deletion'] = (sample_df['LogRRatio'] < -0.2).astype(int)
sample_df['L2R_insertion'] = (sample_df['LogRRatio'] > 0.2).astype(int)

# Group by interval and calculate the mean
results = sample_df.groupby(['NAME', 'CHR', 'START', 'STOP']).agg(
    NUM_VARIANTS=('position', 'count'),
    PERCENT_BAF_INSERTION=('BAF_insertion', 'mean'),
    PERCENT_L2R_DELETION=('L2R_deletion', 'mean'),
    PERCENT_L2R_INSERTION=('L2R_insertion', 'mean')
).reset_index()



# Filter out intervals with less than min_variants
results = results[results['NUM_VARIANTS'] >= min_variants]



In [100]:
import dask.dataframe as dd
from dask import delayed

# Custom function to filter the sample_df based on intervals_df
def filter_by_intervals(partition, intervals_df, kb_window):
    filtered = []
    for _, row in intervals_df.iterrows():
        temp = partition.loc[(partition['chromosome'] == row['CHR']) &
                             (partition['position'] >= row['START'] - kb_window * 1000) &
                             (partition['position'] <= row['STOP'] + kb_window * 1000)]
        temp = temp.assign(NAME=row['NAME'], CHR=row['CHR'], START=row['START'], STOP=row['STOP'])
        filtered.append(temp)
    return pd.concat(filtered)


kb_window = 250
min_variants = 10
min_gentrain = 0.2

metrics_df = dd.read_parquet('/data/GP2/raw_genotypes/snp_metrics/206677820008/snp_metrics_206677820008_R01C01_chr1.parquet')
bim = dd.read_csv('/data/GP2/raw_genotypes/cnvs/release5/genotypes/GP2_round5_APRIL_2023_EUR_clean.bim', sep='\s+', header=None, names=['chr','id','cm','bp','a1','a2'], usecols=['id'])
intervals_df = dd.read_csv('/data/GP2/utils/ref_dir/glist_hg38_chr1.csv', dtype={'CHR':str})

sample_df = metrics_df.loc[(metrics_df.snpID.isin(bim.id.compute())) & (metrics_df.GenTrain_Score >= min_gentrain)]

# Apply the custom filter function to each partition of the sample_df
filtered_sample_df = sample_df.map_partitions(filter_by_intervals, intervals_df=intervals_df, kb_window=kb_window, meta=sample_df.assign(NAME=pd.Series(dtype=object), CHR=pd.Series(dtype=object), START=pd.Series(dtype=int), STOP=pd.Series(dtype=int)))

# Use filtered_sample_df instead of sample_df in the following lines
filtered_sample_df['BAF_insertion'] = ((filtered_sample_df['BAlleleFreq'].between(0.65, 0.85, inclusive='neither')) | (filtered_sample_df['BAlleleFreq'].between(0.15, 0.35, inclusive='neither'))).astype(int)
filtered_sample_df['L2R_deletion'] = (filtered_sample_df['LogRRatio'] < -0.2).astype(int)
filtered_sample_df['L2R_insertion'] = (filtered_sample_df['LogRRatio'] > 0.2).astype(int)

# Group by interval
grouped = filtered_sample_df.groupby(['NAME', 'CHR', 'START', 'STOP'])

# Calculate the aggregations separately
NUM_VARIANTS = grouped['position'].count().rename('NUM_VARIANTS')
PERCENT_BAF_INSERTION = grouped['BAF_insertion'].mean().rename('PERCENT_BAF_INSERTION')
PERCENT_L2R_DELETION = grouped['L2R_deletion'].mean().rename('PERCENT_L2R_DELETION')
PERCENT_L2R_INSERTION = grouped['L2R_insertion'].mean().rename('PERCENT_L2R_INSERTION')

# Concatenate the aggregation results
results = dd.concat([NUM_VARIANTS, PERCENT_BAF_INSERTION, PERCENT_L2R_DELETION, PERCENT_L2R_INSERTION], axis=1).reset_index()

# Filter out intervals with less than min_variants
results = results[results['NUM_VARIANTS'] >= min_variants]
# results.to_parquet(out_path, engine='auto', compression='brotli')


/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/dask/dataframe/multi.py:1254: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [96]:
filtered_sample_df

,chromosome,position,snpID,Sample_ID,Ref,Alt,ALLELE_A,ALLELE_B,BAlleleFreq,LogRRatio,R,Theta,GenTrain_Score,GType,Alt1,Alt2,GT,a1,a2,maf,BAF_insertion,L2R_deletion,L2R_insertion
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,
,object,int64,object,object,object,object,int64,int64,float64,float64,float64,float64,float64,object,object,object,object,object,object,float64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [101]:
# Compute the final result and convert it to a Pandas DataFrame
test_results = results.compute()
test_results

,NAME,CHR,START,STOP,NUM_VARIANTS,PERCENT_BAF_INSERTION,PERCENT_L2R_DELETION,PERCENT_L2R_INSERTION
85,ARHGEF16,1,3454582,3481113,11,0.0000,0.363636,0.000000
160,C1orf101,1,244461370,244640360,10,0.0000,0.000000,0.000000
234,CAMTA1,1,6785323,7769706,10,0.0000,0.000000,0.100000
370,CSMD2,1,33513998,34165842,13,0.0000,0.000000,0.307692
394,DAB1,1,56997905,58250539,12,0.0000,0.083333,0.083333
463,EDARADD,1,236394379,236484708,10,0.0000,0.000000,0.100000
499,ERO1LB,1,236215121,236282039,10,0.0000,0.000000,0.100000
503,ESRRG,1,216503245,217137755,12,0.0000,0.083333,0.083333
582,FHAD1,1,15247271,15398126,10,0.0000,0.200000,0.000000
593,FMN2,1,240091884,240475189,13,0.0000,0.000000,0.076923


In [102]:
sample_test = sample_df.compute()
sample_test.loc[(sample_test['chromosome']=='1') & (sample_test['position'] >= 3204582) &	(sample_test['position']<= 3731113)]

,chromosome,position,snpID,Sample_ID,Ref,Alt,ALLELE_A,ALLELE_B,BAlleleFreq,LogRRatio,R,Theta,GenTrain_Score,GType,Alt1,Alt2,GT,a1,a2,maf
52888,1,3245479,rs74048402,206677820008_R01C01,G,A,1,0,1.021550,-0.216438,1.007590,0.974296,0.825628,AA,A,None,BB,A,G,0.000000
52931,1,3300587,rs2455137,206677820008_R01C01,C,T,1,0,0.494529,0.002118,0.540867,0.513139,0.771683,AB,T,None,AB,T,C,0.500000
52979,1,3345118,rs4415513,206677820008_R01C01,C,T,1,0,0.494152,-0.165281,0.759863,0.582220,0.838088,AB,T,None,AB,T,C,0.500000
52987,1,3353146,rs12076855,206677820008_R01C01,A,C,0,1,-0.011877,-0.287709,0.650681,0.024299,0.852240,AA,C,None,AA,A,C,0.000000
52991,1,3355977,JHU_1.3272540,206677820008_R01C01,G,A,1,0,-0.002809,-0.077267,1.285940,0.018699,0.826347,BB,A,None,AA,A,G,0.437500
53013,1,3384243,rs868688,206677820008_R01C01,C,T,1,0,0.475585,-0.189619,0.881796,0.453435,0.854678,AB,T,None,AB,T,C,0.285714
53017,1,3386882,rs2245703,206677820008_R01C01,T,C,0,1,1.002190,-0.012652,1.065500,0.982750,0.931019,BB,C,None,BB,T,C,0.437500
53300,1,3534441,rs12119711,206677820008_R01C01,C,T,1,0,0.011486,-0.123429,0.772890,0.042202,0.868157,BB,T,None,AA,T,C,0.375000
53350,1,3595370,exm7149,206677820008_R01C01,A,G,0,1,0.049961,-0.330753,0.415949,0.107751,0.791691,AA,G,None,AA,A,G,0.142857
53383,1,3621937,rs4364818,206677820008_R01C01,C,T,1,0,0.440968,0.072807,1.406750,0.447423,0.861270,AB,T,None,AB,T,C,0.357143


In [92]:
sample_test.columns

Index(['chromosome', 'position', 'snpID', 'Sample_ID', 'Ref', 'Alt',
       'ALLELE_A', 'ALLELE_B', 'BAlleleFreq', 'LogRRatio', 'R', 'Theta',
       'GenTrain_Score', 'GType', 'Alt1', 'Alt2', 'GT', 'a1', 'a2', 'maf',
       'CHR', 'START', 'STOP', 'NAME', 'START_PLUS_WINDOW', 'STOP_PLUS_WINDOW',
       'BAF_insertion', 'L2R_deletion', 'L2R_insertion'],
      dtype='object')

In [15]:
missing_cnvs = list()
missing_metrics = list()
for label in ancestry_labels:
    # for label in ['EUR']:
    out_name = f'GP2_round4_JAN_24_2023_{label}'
    label_dir = f'{cnv_path}/cnv_calls/{label}'
    geno_path = f'{cnv_path}/genotypes/{out_name}_clean'
    samples = pd.read_csv(f'{geno_path}_barcode.samples', header=None, names=['IID'])
    
    print(label)
    print(samples.shape[0])
    print()
    print()
    
    
    for sample in samples.IID.unique():
        code = sample.split('_')[0]
        for chrom in chroms:
            
            mfile = f'{snp_metrics_path}/{code}/snp_metrics_{sample}_chr{chrom}.csv'
            cnv_out = f'{label_dir}/CNV_{label}_{sample}_chr{chrom}.csv'
            
            if os.path.isfile(mfile):
                pass
            else:
                missing_metrics.append(mfile)
                
            if os.path.isfile(cnv_out):
                pass
            else:
                missing_cnvs.append(cnv_out)

MDE
120


AMR
489


AJ
1434


AAC
1304


EUR
13611


EAS
154


CAS
313


SAS
79


FIN
11


AFR
404




In [10]:
# write dosages per chromosome per ancestry per cnv-type

with open(f'{swarm_scripts_dir}/cnv_dosages.swarm', 'w') as f:
    for label in ancestry_labels:
        label_dir = f'{cnv_path}/dosages/{label}'
        os.makedirs(label_dir, exist_ok=True)
        for chrom in chroms:

            cnv_files_list = glob.glob(f'{cnv_path}/cnv_calls/{label}/CNV_{label}_*_chr{chrom}.csv')
            cnv_files_df = pd.DataFrame({'filename': cnv_files_list})
            cnv_files_df.to_csv(f'{label_dir}/CNV_{label}_chr{chrom}_files.csv', header=False, index=False)

            for cnv_type in cnv_types:
                
                cmd = f'\
python /data/vitaled2/CNVs/run_cnv_dosage_pipeline.py \
--files {label_dir}/CNV_{label}_chr{chrom}_files.csv \
--label {label} \
--chrom {chrom} \
--cnv_type {cnv_type} \
--out_path {label_dir}/CNV_{label}_chr{chrom}_{cnv_type}.csv'
                f.write(f'{cmd}\n')
f.close()


In [58]:
!swarm -f {swarm_scripts_dir}/cnv_dosages.swarm -g 32 -t 16 --logdir {swarm_scripts_dir}/logs --module python/3.8 --gres=lscratch:20 --partition=norm

58322813


In [67]:
label = 'EUR'
label_dir = f'/data/GP2/raw_genotypes/cnvs/release4/cnv_calls/{label}'
chrom = '1'
cnv_files = f'{cnv_path}/dosages/{label}/CNV_{label}_chr{chrom}_files.csv' 
cnv_type = 'PERCENT_BAF_INSERTION'

cnv_files_df = pd.read_csv(cnv_files, header=None)
files_list = cnv_files_df.loc[:,0]

dosage_ = pd.DataFrame()

for sample_chrom_file in files_list:

    sample = sample_chrom_file.split('/')[-1].replace(f'CNV_{label}_','').replace(f'_chr{chrom}.csv','')
    cnvs = pd.read_csv(sample_chrom_file, usecols=['INTERVAL', cnv_type, 'NUM_VARIANTS'])
    cnvs.loc[:,'sampleid'] = sample
    cnvs_final = cnvs.loc[~cnvs[cnv_type].isna()]
    dosage = cnvs_final.loc[:,[cnv_type,'INTERVAL','sampleid']]
    dosage_pivot = dosage.pivot(index='sampleid', columns='INTERVAL', values=cnv_type)

    dosage_ = pd.concat([dosage_, dosage_pivot])
# dosage_.to_csv(out_path)


In [27]:
# cnv_dosage = pd.read_csv('/data/GP2/raw_genotypes/cnvs/release4/dosages/EUR/CNV_EUR_chr1_PERCENT_BAF_INSERTION.csv')
dose_vc = cnv_dosage.apply(lambda x: x.isnull().value_counts())

In [33]:
dose_vc

,sampleid,A3GALT2,AADACL3,AADACL4,ABCA4,ABCB10,ABCD3,ABL2,ACADM,ACAP3,...,ZNHIT6,ZP4,ZRANB2,ZRANB2-AS1,ZRANB2-AS2,ZSCAN20,ZSWIM5,ZYG11A,ZYG11B,ZZZ3
False,13611.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
True,NaN,13611.0,13611.0,13611.0,13611.0,13611.0,13611.0,13611.0,13611.0,13611.0,...,13611.0,13611.0,13611.0,13611.0,13611.0,13611.0,13611.0,13611.0,13611.0,13611.0


In [84]:
with open(f'{swarm_scripts_dir}/update_cnv_ids.swarm','w') as f:
    for label in ancestry_labels:
        label_dir = f'{cnv_path}/dosages/{label}'
        out_dir = f'/data/GP2/releases/gp2tier2/release4_14022023/cnvs/{label}'
        os.makedirs(out_dir, exist_ok=True)
        for cnv_type in cnv_types:
            for chrom in chroms:

                dosagefile = f'{label_dir}/CNV_{label}_chr{chrom}_{cnv_type}.csv'
                dosagefile_out = f'{out_dir}/CNV_{label}_chr{chrom}_{cnv_type}_gp2ids.csv'
                pheno_out = f'{label_dir}/GP2_{label}_chr{chrom}_{cnv_type}.pheno'
                cmd = f'\
python update_cnv_ids.py \
--dosagefile {dosagefile} \
--key /data/GP2/raw_genotypes/cnvs/release4/key/release4_key.csv \
--pheno_out {pheno_out} \
--out_path {dosagefile_out}'
                f.write(f'{cmd}\n')
f.close()

In [86]:
!swarm -f {swarm_scripts_dir}/update_cnv_ids.swarm -g 8 -t 8 --logdir {swarm_scripts_dir}/logs --module python/3.8 --gres=lscratch:20 --partition=norm

58356484


In [85]:
!head {swarm_scripts_dir}/update_cnv_ids.swarm

python update_cnv_ids.py --dosagefile /data/GP2/raw_genotypes/cnvs/release4/dosages/MDE/CNV_MDE_chr1_PERCENT_BAF_INSERTION.csv --key /data/GP2/raw_genotypes/cnvs/release4/key/release4_key.csv --pheno_out /data/GP2/raw_genotypes/cnvs/release4/dosages/MDE/GP2_MDE_chr1_PERCENT_BAF_INSERTION.pheno --out_path /data/GP2/releases/gp2tier2/release4_14022023/cnvs/MDE/CNV_MDE_chr1_PERCENT_BAF_INSERTION_gp2ids.csv
python update_cnv_ids.py --dosagefile /data/GP2/raw_genotypes/cnvs/release4/dosages/MDE/CNV_MDE_chr2_PERCENT_BAF_INSERTION.csv --key /data/GP2/raw_genotypes/cnvs/release4/key/release4_key.csv --pheno_out /data/GP2/raw_genotypes/cnvs/release4/dosages/MDE/GP2_MDE_chr2_PERCENT_BAF_INSERTION.pheno --out_path /data/GP2/releases/gp2tier2/release4_14022023/cnvs/MDE/CNV_MDE_chr2_PERCENT_BAF_INSERTION_gp2ids.csv
python update_cnv_ids.py --dosagefile /data/GP2/raw_genotypes/cnvs/release4/dosages/MDE/CNV_MDE_chr3_PERCENT_BAF_INSERTION.csv --key /data/GP2/raw_genotypes/cnvs/release4/key/release4_ke

In [66]:
cnv_out_path = '/data/GP2/raw_genotypes/cnvwas'
with open(f'{swarm_scripts_dir}/cnvwas.swarm', 'w') as f:
    for label in ancestry_labels:
        for cnv_type in cnv_types:
            for chrom in chroms:

                dosagefile = f'{cnv_path}/CNV_{label}_chr{chrom}_{cnv_type}_gp2ids.csv'
                pheno = f'{cnv_path}/GP2_{label}_chr{chrom}_{cnv_type}.pheno'
                covar = f'{cnv_path}/GP2_round4_JAN_24_2023_{label}_clean.cov'
                out_path = f'{cnv_out_path}/GP2_{label}_chr{chrom}_{cnv_type}_cnvwas.tab'

                cmd = f'\
python run_cnvwas_pipeline.py \
--cnv_dosage_file {dosagefile} \
--out_path {out_path} \
--pheno {pheno} \
--covar {covar}'
            
                f.write(f'{cmd}\n')
f.close()


In [67]:
!swarm -f {swarm_scripts_dir}/cnvwas.swarm -g 32 -t 16 --time=00:30:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

58077962


In [86]:
from GWAS.gwas import calculate_inflation
label = 'EUR'
# cnv_type = 'PERCENT_BAF_INSERTION'
# cnv_type = 'PERCENT_L2R_DUPLICATION'
# chrom = '1'
pheno_df = pd.read_csv(f'{cnv_path}/GP2_EUR_chr1_{cnv_type}.pheno', sep='\t', engine='python')
ncontrols = pheno_df.pheno.value_counts()[0]
ncases = pheno_df.pheno.value_counts()[1]
cnv_lambda_dict = dict()
for cnv_type in cnv_types:
    total_cnvtype_df = pd.DataFrame()
    for chrom in chroms:
        tab = f'{cnv_out_path}/GP2_{label}_chr{chrom}_{cnv_type}_cnvwas.tab'

        if os.path.isfile(tab):

            sumstats = pd.read_csv(tab, sep='\s+', engine='python')
            # print(tab)
            # infl = calculate_inflation(sumstats.P_VAL, normalize=True, ncases=ncases, ncontrols=ncontrols)
            # print(infl)
            # print()
            # print()
            total_cnvtype_df = pd.concat([total_cnvtype_df, sumstats], ignore_index=True)
        else:
            print(f'{tab} does not exist!')

    infl = calculate_inflation(total_cnvtype_df.P_VAL, normalize=True, ncases=ncases, ncontrols=ncontrols)
    cnv_lambda_dict[cnv_type] = infl
    




RUNNING: lambda_calculation


RUNNING: lambda_calculation


RUNNING: lambda_calculation



In [91]:
cnv_lambda_dict

{'PERCENT_BAF_INSERTION': {'pass': True,
  'step': 'lambda_calculation',
  'metrics': {'inflation': 2.101750589371727}},
 'PERCENT_L2R_DELETION': {'pass': True,
  'step': 'lambda_calculation',
  'metrics': {'inflation': 14.555569305672298}},
 'PERCENT_L2R_DUPLICATION': {'pass': True,
  'step': 'lambda_calculation',
  'metrics': {'inflation': 1.3423052908502526}}}

In [82]:

pheno = pd.read_csv(f'{cnv_path}/GP2_EUR_chr1_{cnv_type}.pheno', sep='\t', engine='python')
ncontrols = pheno.pheno.value_counts()[0]
ncases = pheno.pheno.value_counts()[1]

infl = calculate_inflation(total_cnvtype_df.P_VAL, normalize=True, ncases=ncases, ncontrols=ncontrols)
infl


RUNNING: lambda_calculation



{'pass': True,
 'step': 'lambda_calculation',
 'metrics': {'inflation': 1.3423052908502526}}

In [75]:
ncases

8066

In [65]:
pheno.pheno.value_counts()[1] + pheno.pheno.value_counts()[0] + 1475

13911

1    8066
0    4370
Name: pheno, dtype: int64

In [40]:
fam = pd.read_csv(f'/data/GP2/quality_control/release4/genotype_qc/GP2_round4_JAN_24_2023_{label}.fam', sep='\t', header=None, names=['fid','iid','pat','mat','sex','pheno'], engine='python')
fam.pheno.value_counts()

 2    8711
 1    4876
-9    1475
Name: pheno, dtype: int64

In [54]:

cnv_dosage_file

'/data/GP2/raw_genotypes/cnvs/release4/CNV_EUR_chr1_PERCENT_L2R_DUPLICATION_gp2ids.csv'

In [64]:
import os
import subprocess
import numpy as np
from numpy.core.numeric import NaN
import pandas as pd
import glob
import shutil
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

# Supress copy warning.

pd.options.mode.chained_assignment = None

from QC.utils import shell_do

label = 'EUR'
chrom = '1'
cnv_type = 'PERCENT_L2R_DUPLICATION'
dosagefile = f'{cnv_path}/CNV_{label}_chr{chrom}_{cnv_type}_gp2ids.csv'
pheno = f'{cnv_path}/GP2_{label}_chr{chrom}_{cnv_type}.pheno'
covar = f'{cnv_path}/GP2_round4_JAN_24_2023_{label}_clean.cov'
out_path = f'{cnv_out_path}/GP2_{label}_chr{chrom}_{cnv_type}_cnvwas.tab'
cnv_dosage_file = dosagefile





scaler = MinMaxScaler()
dosage_df = pd.read_csv(cnv_dosage_file)
#fix column names
dosage_df.columns = [x.replace('-','_') for x in dosage_df.columns]
dosage_df.columns = [x.replace('.','_') for x in dosage_df.columns]
dosage_df.columns = [x.replace(' ','_') for x in dosage_df.columns]

pheno_df = pd.read_csv(pheno, sep='\t')
covar_df = pd.read_csv(covar, sep='\t')


if covar_df.age_of_onset.isna().all():
    covar_df.drop(columns=['age_of_onset'], inplace=True)
else:
    covar_df.loc[:,'age_of_onset'] = scaler.fit_transform(covar_df[['age_of_onset']])

if covar_df.age.isna().all():
    covar_df.drop(columns=['age'], inplace=True)
else:
    covar_df.loc[:,'age'] = scaler.fit_transform(covar_df[['age']])

if covar_df.sex.isna().all():
    covar_df.drop(columns=['sex'], inplace=True)

covar_df.drop(columns=['FID'], inplace=True)
covar_df.rename(columns={'GP2sampleID':'sampleid'}, inplace=True)

data_df = dosage_df.merge(covar_df, on='sampleid', how='left').merge(pheno_df, on='sampleid', how='left').set_index('sampleid')

rm_pred = [f'PC{i}' for i in range(1,21)] + ['sex','age_of_onset','age','pheno']

pred_list = [x for x in data_df.columns if x not in rm_pred]
covars_list = [x for x in data_df.columns if x not in pred_list + [f'PC{i}' for i in range(11,21)] + ['pheno']]


genes_list = dosage_df.columns.drop('sampleid')
data_df_final = data_df.copy()
ctrl_stds = data_df.loc[data_df['pheno']==0, genes_list].std()
ctrl_means = data_df.loc[data_df['pheno']==0, genes_list].mean()

for gene in genes_list:
    lower = (ctrl_means[gene]-(ctrl_stds[gene]*2))
    upper = (ctrl_means[gene]+(ctrl_stds[gene]*2))         
    data_df_final.loc[:, gene] = data_df_final.loc[:, gene].between(lower, upper, inclusive='both')
    
data_df_final.loc[:, genes_list] = data_df_final.loc[:, genes_list].astype(int)

results = []
fails = []

for pred in range(len(pred_list)):
    pred_name = pred_list[pred]
    formula = "pheno ~ " + pred_name + " + " + ' + '.join(covars_list)

    fitted = sm.formula.glm(formula=formula, family=sm.families.Binomial(), data=data_df_final).fit()

    beta_coef  = fitted.params.loc[pred_name]
    beta_se  = fitted.bse.loc[pred_name]
    p_val = fitted.pvalues.loc[pred_name]

    results.append((pred_name, beta_coef, beta_se, p_val))


output = pd.DataFrame(results, columns=('PREDICTOR', 'BETA_COEF', 'BETA_SE','P_VAL'))
# output.to_csv(out_path, sep='\t', header=True, index=False)
    















# def CNV_WAS(cnv_dosage_file, pheno, covar, out_path):
#     scaler = MinMaxScaler()
#     dosage_df = pd.read_csv(cnv_dosage_file)
#     #fix column names
#     dosage_df.columns = [x.replace('-','_') for x in dosage_df.columns]
#     dosage_df.columns = [x.replace('.','_') for x in dosage_df.columns]
#     dosage_df.columns = [x.replace(' ','_') for x in dosage_df.columns]

#     pheno_df = pd.read_csv(pheno, sep='\t')
#     covar_df = pd.read_csv(covar, sep='\t')


#     if covar_df.age_of_onset.isna().all():
#         covar_df.drop(columns=['age_of_onset'], inplace=True)
#     else:
#         covar_df.loc[:,'age_of_onset'] = scaler.fit_transform(covar_df[['age_of_onset']])

#     if covar_df.age.isna().all():
#         covar_df.drop(columns=['age'], inplace=True)
#     else:
#         covar_df.loc[:,'age'] = scaler.fit_transform(covar_df[['age']])

#     if covar_df.sex.isna().all():
#         covar_df.drop(columns=['sex'], inplace=True)

#     covar_df.drop(columns=['FID'], inplace=True)
#     covar_df.rename(columns={'GP2sampleID':'sampleid'}, inplace=True)

#     data_df = dosage_df.merge(covar_df, on='sampleid', how='left').merge(pheno_df, on='sampleid', how='left').set_index('sampleid')

#     rm_pred = [f'PC{i}' for i in range(1,21)] + ['sex','age_of_onset','age','pheno']

#     pred_list = [x for x in data_df.columns if x not in rm_pred]
#     covars_list = [x for x in data_df.columns if x not in pred_list + [f'PC{i}' for i in range(11,21)] + ['pheno']]


#     genes_list = dosage_df.columns.drop('sampleid')
#     data_df_final = data_df.copy()
#     ctrl_stds = data_df.loc[data_df['pheno']==0, genes_list].std()
#     ctrl_means = data_df.loc[data_df['pheno']==0, genes_list].mean()

#     for gene in genes_list:
#         lower = (ctrl_means[gene]-(ctrl_stds[gene]*2))
#         upper = (ctrl_means[gene]+(ctrl_stds[gene]*2))         
#         data_df_final.loc[:, gene] = data_df_final.loc[:, gene].between(lower, upper, inclusive='both')

#     data_df_final.loc[:, genes_list] = data_df_final.loc[:, genes_list].astype(int)

#     results = []
#     fails = []

#     for pred in range(len(pred_list)):
#         pred_name = pred_list[pred]
#         formula = "pheno ~ " + pred_name + " + " + ' + '.join(covars_list)

#         fitted = sm.formula.glm(formula=formula, family=sm.families.Binomial(), data=data_df_final).fit()
#         beta_coef  = fitted.params.loc[pred_name]
#         beta_se  = fitted.bse.loc[pred_name]
#         p_val = fitted.pvalues.loc[pred_name]

#         results.append((pred_name, beta_coef, beta_se, p_val))


#     output = pd.DataFrame(results, columns=('PREDICTOR', 'BETA_COEF', 'BETA_SE','P_VAL'))
#     output.to_csv(out_path, sep='\t', header=True, index=False)

Optimization terminated successfully.
         Current function value: 0.638843
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.639250
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.639028
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.639249
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.639250
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.639149
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.636706
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.639198
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.639250
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.639085
  

In [65]:
from GWAS.gwas import calculate_inflation
# pheno_df = pd.read_csv(f'{cnv_path}/GP2_EUR_chr1_{cnv_type}.pheno', sep='\t', engine='python')
ncontrols = pheno_df.pheno.value_counts()[0]
ncases = pheno_df.pheno.value_counts()[1]

infl = calculate_inflation(output.P_VAL, normalize=True, ncases=ncases, ncontrols=ncontrols)
infl



RUNNING: lambda_calculation



{'pass': True,
 'step': 'lambda_calculation',
 'metrics': {'inflation': 1.4366622566944616}}

In [49]:
formula

'pheno ~ WDR64 + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + sex + age + age_of_onset'

In [44]:

data_df_final

,ACTN2,AGTRAP,ARHGEF16,C1orf101,C1orf167,CAMTA1,CSMD2,DAB1,DRAXIN,EDARADD,...,PC15,PC16,PC17,PC18,PC19,PC20,sex,age,age_of_onset,pheno
sampleid,,,,,,,,,,,,,,,,,,,,,
CORIELL_000178_s1,1,1,1,1,1,1,1,1,1,1,...,-0.006651,0.009075,-0.007526,0.000607,-0.005277,0.016480,1,0.646341,0.600000,1
PAGE_000432_s1,1,1,1,1,1,1,0,0,1,1,...,-0.009945,0.006582,0.005732,-0.005267,-0.017189,-0.012584,1,0.767308,0.741228,1
CORIELL_000783_s1,1,1,1,1,1,1,1,1,1,1,...,0.001914,-0.002026,-0.007720,-0.002451,-0.003864,-0.006665,2,0.524390,0.578947,1
CORIELL_004856_s1,1,1,1,1,1,1,1,1,1,1,...,0.006815,-0.005230,-0.006799,0.006651,0.005691,-0.003396,1,0.621951,0.577225,0
COPN_000306_s1,1,1,1,1,1,1,1,1,1,1,...,0.004314,0.003814,0.000499,0.010819,0.001542,0.018862,1,0.548780,0.577225,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PAGE_001054_s1,1,1,1,1,1,1,1,1,1,1,...,-0.008115,-0.003489,0.001157,-0.002412,-0.000864,-0.004991,1,0.722300,0.653509,1
LCC_000146_s1,1,1,1,1,1,1,1,1,1,1,...,0.008954,-0.009757,0.003442,-0.000514,-0.001066,-0.003567,1,0.548780,0.577225,0
CORIELL_002459_s1,1,1,1,1,1,1,1,1,1,1,...,0.001957,0.003340,0.000891,-0.007043,-0.000739,0.000489,1,0.536585,0.484211,1


In [13]:
# output final release samples
cnv_out_path = '/data/CARD/PD/GP2/genotypes/GP2/round3/GP2_cnvs'
release3_samples = '/data/CARD/PD/GP2/genotypes/GP2/round3/clean/release3.samples'
release_key_final = pd.read_csv('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/release3_master_key.csv')
release_key_out = release_key_final.loc[release_key_final.pheno!=-9]
release_key_out['GP2sampleID'].to_csv(release3_samples, index=False, header=False)

In [20]:
with open(f'{swarm_scripts_dir}/cnv_dosage_release_split.swarm', 'w') as f:
    for label in ancestry_labels:
        sample_file = f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/{label}_release3.samples'
        label_samples = release_key_final.loc[release_key_final['label']==label]
        label_samples['GP2sampleID'].to_csv(sample_file, header=False, index=False)

        for cnv_type in cnv_types:
            for chrom in chroms:
                
                dosagefile = f'{cnv_path}/CNV_{label}_chr{chrom}_{cnv_type}_gp2ids.csv'
                dosage_out = f'{cnv_out_path}/CNV_{label}_chr{chrom}_{cnv_type}_release3.csv'
                
                cmd = f'\
python split_release_samples.py \
--cnv_dosage_file {dosagefile} \
--samples {sample_file} \
--out_path {dosage_out}'
            
                f.write(f'{cmd}\n')
f.close()


In [21]:
!swarm -f {swarm_scripts_dir}/cnv_dosage_release_split.swarm -g 8 -t 8 --time=00:30:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

53715605


In [14]:
# CHECK COUNTS
total_dosages = 0
for label in ancestry_labels:
    for cnv_type in cnv_types:
        dosage_out = f'{cnv_out_path}/CNV_{label}_chr1_{cnv_type}_release3.csv'
        print(label, cnv_type)
        label_count = !cat {dosage_out} | wc -l
        label_count = int(label_count[0])
        total_dosages += label_count
        print(label_count)
        print()

AMR PERCENT_BAF_INSERTION
412

AMR PERCENT_L2R_DELETION
412

AMR PERCENT_L2R_DUPLICATION
412

EUR PERCENT_BAF_INSERTION
10727

EUR PERCENT_L2R_DELETION
10727

EUR PERCENT_L2R_DUPLICATION
10727

SAS PERCENT_BAF_INSERTION
59

SAS PERCENT_L2R_DELETION
59

SAS PERCENT_L2R_DUPLICATION
59

MDE PERCENT_BAF_INSERTION
62

MDE PERCENT_L2R_DELETION
62

MDE PERCENT_L2R_DUPLICATION
62

EAS PERCENT_BAF_INSERTION
137

EAS PERCENT_L2R_DELETION
137

EAS PERCENT_L2R_DUPLICATION
137

AAC PERCENT_BAF_INSERTION
1215

AAC PERCENT_L2R_DELETION
1215

AAC PERCENT_L2R_DUPLICATION
1215

AFR PERCENT_BAF_INSERTION
404

AFR PERCENT_L2R_DELETION
404

AFR PERCENT_L2R_DUPLICATION
404

CAS PERCENT_BAF_INSERTION
303

CAS PERCENT_L2R_DELETION
303

CAS PERCENT_L2R_DUPLICATION
303

AJ PERCENT_BAF_INSERTION
923

AJ PERCENT_L2R_DELETION
923

AJ PERCENT_L2R_DUPLICATION
923



In [22]:
# push to release bucket
with open(f'{swarm_scripts_dir}/gcp_push_cnvs.swarm','w') as f:
    
    for label in ancestry_labels:
        for chrom in chroms:
    
#         !module load google-cloud-sdk/397.0.0; gsutil cp {cnv_out_path}/CNV_{label}_BAF_INSERTION_release3.csv gs://gp2tier2/release3_31102022/cnvs/{label}/
            cp1 = f'gsutil cp {cnv_out_path}/CNV_{label}_chr{chrom}_PERCENT_BAF_INSERTION_release3.csv gs://gp2tier2/release3_31102022/cnvs/{label}/'
            cp2 = f'gsutil cp {cnv_out_path}/CNV_{label}_chr{chrom}_PERCENT_L2R_DELETION_release3.csv gs://gp2tier2/release3_31102022/cnvs/{label}/'
            cp3 = f'gsutil cp {cnv_out_path}/CNV_{label}_chr{chrom}_PERCENT_L2R_DUPLICATION_release3.csv gs://gp2tier2/release3_31102022/cnvs/{label}/'

            cps = [cp1, cp2, cp3]

            for cp in cps:
                f.write(f'{cp}\n')
f.close()

In [23]:
!swarm -f {swarm_scripts_dir}/gcp_push_cnvs.swarm -g 8 -t 8 --time=00:30:00 --logdir {swarm_scripts_dir}/logs --module google-cloud-sdk/397.0.0 --gres=lscratch:20 --partition=norm

53719849


In [24]:
!module load google-cloud-sdk/397.0.0; gsutil cp /data/CARD/PD/GP2/genotypes/GP2/round3/GP2_cnvs/CNV_AMR_chr1_BAF_INSERTION_release3.csv gs://gp2tier2/release3_31102022/cnvs/AMR/

594


In [24]:
!ls -lh {cnv_out_path}/CNV_{label}_chr{chrom}_PERCENT_BAF_INSERTION_release3.csv

-rw-rw----+ 1 vitaled2 CARD 4.2K Dec  5 19:27 /data/CARD/PD/GP2/genotypes/GP2/round3/GP2_cnvs/CNV_AJ_chr22_PERCENT_BAF_INSERTION_release3.csv


In [15]:
!cat /data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/CNV_EUR_chr1_PERCENT_BAF_INSERTION_gp2ids.csv | wc -l

11696


In [ ]:
{cnv_path}/genotypes/